https://medium.com/analytics-vidhya/recurrent-neural-networks-rnns-and-time-series-forecasting-d9ea933426b3

# Fundamentos de RNN y LSTM con Keras

### import libraries

In [1]:
import numpy as np
import pandas as pd
from statsmodels.tsa.statespace.sarimax import SARIMAX
from statsmodels.graphics.tsaplots import plot_acf,plot_pacf 
from statsmodels.tsa.seasonal import seasonal_decompose 
import matplotlib.pyplot as plt                    
from sklearn.metrics import mean_squared_error
from statsmodels.tools.eval_measures import rmse
import warnings
import matplotlib.pyplot as plt
warnings.filterwarnings("ignore")

from keras.models import Sequential
from keras.layers import LSTM, Dense
from tensorflow.keras.preprocessing.sequence import TimeseriesGenerator

### generate_time_series

In [2]:
#imports the numpy module and renames it np for easier access.
import numpy as np

#defines a function called generate_time_series that takes two arguments: batch_size and n_steps
def generate_time_series(batch_size, n_steps):
    
    #generates four arrays of random numbers between 0 and 1. Each array has a form of (batch_size, 1)
    freq1, freq2, offsets1, offsets2 = np.random.rand(4, batch_size, 1)
    
    #generates a sequence of n_steps numbers equally spaced between 0 and 1
    time = np.linspace(0, 1, n_steps)
    
    #generates a time series based on a sine function. The frequency and phase of the sine function are determined by 
    #freq1 and offsets1, respectively.
    series = 0.5 * np.sin((time - offsets1) * (freq1 * 10 + 10))    # +wave 1
    
    #add another time series based on a sine function to the existing series. The frequency and phase of this sine 
    #function are determined by freq2 and offsets2, respectively
    series += 0.2 * np.sin((time - offsets2) * (freq2 * 20 + 20))            
    
    #adds random noise to the time series. The noise is an array of random numbers between -0.5 and 0.5.
    series += 0.1 * (np.random.rand(batch_size, n_steps) - 0.5)    
    
    #devuelve la serie de tiempo con una dimensión adicional y convierte los datos a float32
    return series[..., np.newaxis].astype(np.float32)

In [4]:
#defines a variable n_steps which is set to 50. This variable will be used to determine the number of time steps 
#in each time series.
n_steps = 50

#call the generate_time_series function you defined above, generating 10,000 time series, each with n_steps + 1 time steps.
series = generate_time_series(10000, n_steps + 1)

#divides the time series into training sets. X_train contains the first n_steps of the first 7000 series, and y_train 
#contains the last time step of the same 7000 series.
X_train, y_train = series[:7000, :n_steps], series[:7000, -1]

#divides time series into validation sets. X_valid contains the first n_steps of the series 7000 to 9000, and y_valid 
#contains the last time step of the same series.
X_valid, y_valid = series[7000:9000, :n_steps], series[7000:9000, -1]

#divides time series into test sets. X_test contains the first n_steps of series 9000 onwards, and y_test contains 
#the last time step of the same series
X_test, y_test = series[9000:, :n_steps], series[9000:, -1]

## RNN Simple

In [3]:
import tensorflow as tf
from tensorflow import keras

In [5]:
import tensorflow as tf
from tensorflow import keras

# Definición del model
model = keras.models.Sequential([
    #Añadimos una capa de RNN con una neurona
    keras.layers.SimpleRNN(1, input_shape=[None, 1])
])

# Definimos el optimizador, en este caso de Adam 
optimizer = keras.optimizers.Adam()

# Generamos el modelo
model.compile(loss="mse", 
              optimizer=optimizer)

# Entrenamos el modelo
history = model.fit(X_train, y_train, epochs=20,
                    validation_data=(X_valid, y_valid))

# Evaluación del modelo
model.evaluate(X_valid, y_valid)

Epoch 1/20
219/219 ━━━━━━━━━━━━━━━━━━━━ 11s 13ms/step - loss: 0.0423 - val_loss: 0.0286
Epoch 2/20
219/219 ━━━━━━━━━━━━━━━━━━━━ 2s 10ms/step - loss: 0.0246 - val_loss: 0.0184
Epoch 3/20
219/219 ━━━━━━━━━━━━━━━━━━━━ 2s 10ms/step - loss: 0.0162 - val_loss: 0.0137
Epoch 4/20
219/219 ━━━━━━━━━━━━━━━━━━━━ 2s 10ms/step - loss: 0.0128 - val_loss: 0.0121
Epoch 5/20
219/219 ━━━━━━━━━━━━━━━━━━━━ 2s 10ms/step - loss: 0.0113 - val_loss: 0.0117
Epoch 6/20
219/219 ━━━━━━━━━━━━━━━━━━━━ 2s 10ms/step - loss: 0.0115 - val_loss: 0.0116
Epoch 7/20
219/219 ━━━━━━━━━━━━━━━━━━━━ 2s 10ms/step - loss: 0.0111 - val_loss: 0.0116
Epoch 8/20
219/219 ━━━━━━━━━━━━━━━━━━━━ 2s 9ms/step - loss: 0.0114 - val_loss: 0.0116
Epoch 9/20
219/219 ━━━━━━━━━━━━━━━━━━━━ 2s 10ms/step - loss: 0.0113 - val_loss: 0.0116
Epoch 10/20
219/219 ━━━━━━━━━━━━━━━━━━━━ 2s 10ms/step - loss: 0.0111 - val_loss: 0.0116
Epoch 11/20
219/219 ━━━━━━━━━━━━━━━━━━━━ 2s 10ms/step - loss: 0.0110 - val_loss: 0.0116
Epoch 12/20
219/219 ━━━━━━━━━━━━━━━━━━━━ 

0.011577751487493515

## RNN profunda

In [6]:
# Definimos un modelo de RNN con múltiples capas

model = keras.models.Sequential([
    keras.layers.SimpleRNN(10, return_sequences=True, input_shape=[None, 1]),
    keras.layers.SimpleRNN(10, return_sequences=True),
    keras.layers.SimpleRNN(1)
])

model.compile(loss="mse", optimizer="adam")

history = model.fit(X_train, y_train, epochs=20,
                    validation_data=(X_valid, y_valid))

model.evaluate(X_valid, y_valid)

Epoch 1/20
219/219 ━━━━━━━━━━━━━━━━━━━━ 13s 29ms/step - loss: 0.2032 - val_loss: 0.0233
Epoch 2/20
219/219 ━━━━━━━━━━━━━━━━━━━━ 5s 25ms/step - loss: 0.0201 - val_loss: 0.0133
Epoch 3/20
219/219 ━━━━━━━━━━━━━━━━━━━━ 5s 25ms/step - loss: 0.0111 - val_loss: 0.0084
Epoch 4/20
219/219 ━━━━━━━━━━━━━━━━━━━━ 6s 25ms/step - loss: 0.0076 - val_loss: 0.0062
Epoch 5/20
219/219 ━━━━━━━━━━━━━━━━━━━━ 6s 26ms/step - loss: 0.0061 - val_loss: 0.0051
Epoch 6/20
219/219 ━━━━━━━━━━━━━━━━━━━━ 6s 28ms/step - loss: 0.0052 - val_loss: 0.0046
Epoch 7/20
219/219 ━━━━━━━━━━━━━━━━━━━━ 6s 25ms/step - loss: 0.0047 - val_loss: 0.0045
Epoch 8/20
219/219 ━━━━━━━━━━━━━━━━━━━━ 6s 25ms/step - loss: 0.0044 - val_loss: 0.0040
Epoch 9/20
219/219 ━━━━━━━━━━━━━━━━━━━━ 6s 25ms/step - loss: 0.0041 - val_loss: 0.0039
Epoch 10/20
219/219 ━━━━━━━━━━━━━━━━━━━━ 6s 26ms/step - loss: 0.0040 - val_loss: 0.0037
Epoch 11/20
219/219 ━━━━━━━━━━━━━━━━━━━━ 6s 26ms/step - loss: 0.0038 - val_loss: 0.0038
Epoch 12/20
219/219 ━━━━━━━━━━━━━━━━━━━━

0.0036206489894539118

## LSTM

In [8]:
# Definimos un modelo básico de LSTM

model = Sequential()
model.add(LSTM(100, activation='relu', input_shape=(n_input, n_features)))
model.add(Dense(1))

model.compile(optimizer='adam', loss='mse')

model.fit_generator(generator,epochs=90)